In [1]:
import sys
import os

# Go two levels up to reach the root folder "ResearchAgent"
root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(root_path)


In [2]:
import os
print("Current working directory:", os.getcwd())


Current working directory: d:\Projects\Research Agent\Graph


In [4]:
from langgraph.graph import StateGraph, START, END
from Graph.Nodes.state import ResearchState
from Graph.Nodes.compile_results_node import compile_results_node
from Graph.Nodes.hypothesis_generation_node import hypothesis_generation_node
from Graph.Nodes.literature_search_node import literature_search_node
from Graph.Nodes.summarization_node import summarization_node

In [5]:
from utils.loadapi import loadapi
loadapi()

In [6]:
def should_continue(state: ResearchState) -> str:
    """Decide if we should continue or end due to errors."""
    if state.errors:
        return "handle_errors"
    return "continue"

def handle_errors_node(state: ResearchState) -> ResearchState:
    """Create an error report."""
    error_report = "## Errors Encountered\n\n"
    for error in state.errors:
        error_report += f"- {error}\n"
    
    state.final_output = error_report
    return state

In [7]:
def create_research_workflow():
    """Create and return a LangGraph workflow for the research Process."""
    graph = StateGraph(ResearchState)
    
    # Add nodes
    graph.add_node("literature_search_node", literature_search_node)
    graph.add_node("summarization_node", summarization_node)
    graph.add_node("hypothesis_generation_node", hypothesis_generation_node)
    graph.add_node("compile_results_node", compile_results_node)
    graph.add_node("handle_errors_node", handle_errors_node)
    
    # Add edges
    graph.add_edge(START, "literature_search_node")
    graph.add_edge("literature_search_node", "summarization_node")
    graph.add_edge("summarization_node", "hypothesis_generation_node")
    graph.add_edge("hypothesis_generation_node", "compile_results_node")
    graph.add_edge("compile_results_node", END)
    graph.add_edge("handle_errors_node", END)
    
    # Add conditionanl Edges
    graph.add_conditional_edges(
        "literature_search_node",
        should_continue,
        {
            "continue": "summarization_node",
            "handle_errors": "handle_errors_node"
        }
    )
    
    graph.add_conditional_edges(
        "summarization_node",
        should_continue,
        {
            "continue": "hypothesis_generation_node",
            "handle_errors": "handle_errors_node"
        }
    )
    
    graph.add_conditional_edges(
        "hypothesis_generation_node",
        should_continue,
        {
            "continue": "compile_results_node",
            "handle_errors": "handle_errors_node"
        }
    )
    
    return graph

graph = create_research_workflow()
compiled_graph = graph.compile()

In [8]:
from IPython.display import Image, display

In [9]:
print(compiled_graph.get_graph().draw_ascii())

                                            +-----------+                             
                                            | __start__ |                             
                                            +-----------+                             
                                                  *                                   
                                                  *                                   
                                                  *                                   
                                     +------------------------+                       
                                     | literature_search_node |                       
                                     +------------------------+..                     
                                         **                      ......               
                                      ***                              ......         
                                    **     

In [10]:
print(compiled_graph.get_graph().draw_mermaid())

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	literature_search_node(literature_search_node)
	summarization_node(summarization_node)
	hypothesis_generation_node(hypothesis_generation_node)
	compile_results_node(compile_results_node)
	handle_errors_node(handle_errors_node)
	__end__([<p>__end__</p>]):::last
	__start__ --> literature_search_node;
	compile_results_node --> __end__;
	handle_errors_node --> __end__;
	hypothesis_generation_node --> compile_results_node;
	literature_search_node --> summarization_node;
	summarization_node --> hypothesis_generation_node;
	literature_search_node -. &nbsp;continue&nbsp; .-> summarization_node;
	literature_search_node -. &nbsp;handle_errors&nbsp; .-> handle_errors_node;
	summarization_node -. &nbsp;continue&nbsp; .-> hypothesis_generation_node;
	summarization_node -. &nbsp;handle_errors&nbsp; .-> handle_errors_node;
	hypothesis_generation_node -. &nbsp;continue&nbsp; .-> compile_results_node;
	hypo

In [11]:
import nest_asyncio
nest_asyncio.apply()

In [12]:
from langchain_core.runnables.graph_mermaid import MermaidDrawMethod

# Use the Pyppeteer rendering method
display(Image(compiled_graph.get_graph().draw_mermaid_png(draw_method=MermaidDrawMethod.PYPPETEER)))

[INFO] Starting Chromium download.


OSError: Chromium downloadable not found at https://storage.googleapis.com/chromium-browser-snapshots/Win_x64/1181205/chrome-win.zip: Received <?xml version='1.0' encoding='UTF-8'?><Error><Code>NoSuchKey</Code><Message>The specified key does not exist.</Message><Details>No such object: chromium-browser-snapshots/Win_x64/1181205/chrome-win.zip</Details></Error>.


In [13]:
# Method 2: Use draw_mermaid() to get just the text syntax
mermaid_syntax = compiled_graph.get_graph().draw_mermaid()
print(mermaid_syntax)

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	literature_search_node(literature_search_node)
	summarization_node(summarization_node)
	hypothesis_generation_node(hypothesis_generation_node)
	compile_results_node(compile_results_node)
	handle_errors_node(handle_errors_node)
	__end__([<p>__end__</p>]):::last
	__start__ --> literature_search_node;
	compile_results_node --> __end__;
	handle_errors_node --> __end__;
	hypothesis_generation_node --> compile_results_node;
	literature_search_node --> summarization_node;
	summarization_node --> hypothesis_generation_node;
	literature_search_node -. &nbsp;continue&nbsp; .-> summarization_node;
	literature_search_node -. &nbsp;handle_errors&nbsp; .-> handle_errors_node;
	summarization_node -. &nbsp;continue&nbsp; .-> hypothesis_generation_node;
	summarization_node -. &nbsp;handle_errors&nbsp; .-> handle_errors_node;
	hypothesis_generation_node -. &nbsp;continue&nbsp; .-> compile_results_node;
	hypo

In [14]:
from langchain_core.runnables.graph_mermaid import MermaidDrawMethod

# Try the pyppeteer method which uses a local browser
display(Image(compiled_graph.get_graph().draw_mermaid_png(
    draw_method=MermaidDrawMethod.PYPPETEER
)))

[INFO] Starting Chromium download.


OSError: Chromium downloadable not found at https://storage.googleapis.com/chromium-browser-snapshots/Win_x64/1181205/chrome-win.zip: Received <?xml version='1.0' encoding='UTF-8'?><Error><Code>NoSuchKey</Code><Message>The specified key does not exist.</Message><Details>No such object: chromium-browser-snapshots/Win_x64/1181205/chrome-win.zip</Details></Error>.


In [15]:
display(Image(compiled_graph.get_graph().draw_mermaid_png()))

ReadTimeout: HTTPSConnectionPool(host='mermaid.ink', port=443): Read timed out. (read timeout=10)

In [16]:
display(Image(compiled_graph.get_graph().draw_mermaid_png()))

ReadTimeout: HTTPSConnectionPool(host='mermaid.ink', port=443): Read timed out. (read timeout=10)